In [11]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import pandas as pd
from func_timeout import func_timeout, FunctionTimedOut, func_set_timeout

In [16]:
internal_urls = set()
external_urls = set()

def is_valid(url):
    parsed = urlparse(url)
    
    if parsed.scheme.startswith('http') and bool(parsed.netloc):
        return True
    else:
        return False

@func_set_timeout(10)
def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    
    total_internal = 0
    total_external = 0
        
    for a_tag in soup.findAll("a"):
        
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            # href empty tag
            continue
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
            
        if href in internal_urls:
            total_internal = total_internal + 1
            continue
            
        if domain_name not in href:
            total_external = total_external + 1
            if href not in external_urls:
                external_urls.add(href)
            continue
            
        urls.add(href)
        internal_urls.add(href)
        total_internal = total_internal + 1
                
    return total_internal, total_external

In [13]:
# df = pd.DataFrame({
#     'Url':['https://www.bouwmaat.nl/bouwmaterialen','https://www.hornbach.nl/shop/Bouwmateriaal/S4471/artikeloverzicht.html','https://www.bouwbestel.nl/bouwmaterialen.html'],
#     'Internal':[0,0,0],
#     'External':[0,0,0]
# })

In [14]:
def loop_get_all_website_links(dataframe):
    InternalLinks = []
    ExternalLinks = []
    for url in dataframe.index:
        try:
            var = get_all_website_links(dataframe['Ur'][url])
        except:
            var = [None, None]
        InternalLinks.append(var[0])
        ExternalLinks.append(var[1])
    return InternalLinks, ExternalLinks

# counter = 0
# for i in df.Url:
#     gadget = get_all_website_links(i)
#     df.Internal[counter] = gadget[0]
#     df.External[counter] = gadget[1]
#     counter = counter + 1

In [15]:
# dataframe = pd.read_csv("calls\dataframe-73f2a3dfe1f688c31ab0ae26a247078465169f60f9a383c28fc900bc7d01a132.csv")

print(loop_get_all_website_links(dataframe))

([1425, 217, 2595, 138, 563, 1029, 0, 615, 138, 348, 440, 349, 297, 24, 87, 784, 70, 158, 299, 162, 117, 64, 828, 518, 598, 34, 109, 62, 50, 197, 134, 59, 371, 276, 400, 28, None, 229, 0, 61, 371, 109, 487, 85, 99, 128, 197, 24, 1504, 713], [11, 7, 14, 10, 5, 3, 0, 5, 9, 16, 5, 3, 13, 6, 10, 16, 10, 6, 13, 70, 0, 1, 16, 14, 11, 4, 1, 3, 10, 7, 52, 6, 6, 45, 10, 3, None, 3, 0, 14, 9, 10, 6, 9, 18, 14, 7, 2, 7, 3])
